In [1]:
import numpy as np
import skimage
import cv2
import nd2
import xarray
from aicspylibczi import CziFile
from aicsimageio import AICSImage
import gc

# DL frameworks
from cellpose import models
from spotiflow.model import Spotiflow

# figures
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

# plotting and processing
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import warnings
import openpyxl

ImportError: /home/michalv/miniforge3/envs/cellpose-bags/lib/python3.11/site-packages/cv2/python-3.11/../../../../libopencv_imgcodecs.so.411: undefined symbol: _ZN7Imf_3_36HeaderC1EiifRKN9Imath_3_14Vec2IfEEfNS_9LineOrderENS_11CompressionE

In [ ]:
experiment_folder = Path.cwd().parent 
raw_data_folder = experiment_folder / 'raw_data'
figures_folder = experiment_folder / 'figures'
processed_data_folder = experiment_folder / 'processed_data'
os.environ["CELLPOSE_LOCAL_MODELS_PATH"] = str(experiment_folder.parent / '_pipeline_assets/cellpose_models/') # location of cellpose models


# Create output folders if they don't exist
figures_folder.mkdir(exist_ok=True)
processed_data_folder.mkdir(exist_ok=True)

# ANALYSIS PARAMETERS
thickness_um = 20

# Channel order
channels_params = {
    'brightfield': 0,
    'spots': 1,
    'bags': 2
}

# Segmentation parameters
segmentation_params = {
    'model_name': 'cpsam_20x_downsampeled_20250630',
    'downsample_factor': 0.1
}

footprint = np.array([
    [1,0,1,0,1],
    [0,1,1,1,0],
    [1,1,1,1,1],
    [0,1,1,1,0],
    [1,0,1,0,1]
], dtype=bool)

print(f"Experiment folder: {experiment_folder}")
print(f"Gel thickness: {thickness_um} μm")

In [ ]:
def stdev_project_xarray(array):
    """Standard deviation projection of single xarray channel"""
    return np.std(array.values, axis=0)

def process_field_of_view(array, p, channels_params):
    seg_data = array.isel(P=p, C=channels_params['brightfield'])
    spots_data = array.isel(P=p, C=channels_params['spots'])
    
    seg_projected = stdev_project_xarray(seg_data)
    del seg_data
    
    spots_projected = stdev_project_xarray(spots_data)
    del spots_data
    
    return seg_projected, spots_projected

# segmentation
def cellpose_bag(image):
    """Run bag pretrained cellpose SAM"""
    model = models.CellposeModel(
        gpu=True,
        pretrained_model=segmentation_params['model_name']
    )
    
    masks, flows, styles = model.eval(
        image
    )
    return masks

def downsample(image: np.ndarray, scale_factor: float) -> np.ndarray:
    if not 0.0 < scale_factor <= 1.0:
        raise ValueError("scale_factor must be between 0.0 and 1.0.")

    original_height, original_width = image.shape[:2]
    new_width = int(original_width * scale_factor)
    new_height = int(original_height * scale_factor)
    new_dimensions = (new_width, new_height)

    downscaled_image = cv2.resize(
        image,
        new_dimensions,
        interpolation=cv2.INTER_AREA
    )

    return downscaled_image

def upsample_mask(mask: np.ndarray, original_shape: tuple) -> np.ndarray:
    original_height, original_width = original_shape[:2]
    target_dimensions = (original_width, original_height) # cv2 uses (width, height)

    upscaled_mask = cv2.resize(
        mask,
        target_dimensions,
        interpolation=cv2.INTER_NEAREST
    )

    return upscaled_mask

def memory_efficient_segmentation(image, segmentation_params):
    downsampled = downsample(image, scale_factor=segmentation_params['downsample_factor'])
    
    masks = cellpose_bag(downsampled)
    del downsampled
    
    upsampled_masks = upsample_mask(masks, image.shape)
    del masks
    
    filtered_masks = skimage.segmentation.clear_border(upsampled_masks, buffer_size=10)
    del upsampled_masks
    
    return filtered_masks

# spots
def spot_tophat_correction(image, footprint):
    corrected_spots = skimage.morphology.white_tophat(image, footprint=footprint)
    return corrected_spots

def detect_spots_spotiflow(image):
    """Detect spots spotiflow"""
    model = Spotiflow.from_pretrained("general")
    points, details = model.predict(
        img=image,
        verbose=False
    )
    return points, details


def assign_spots_to_mask(coordinates, mask):
    if len(coordinates) == 0:
        return []
    
    # Convert to numpy array if needed
    if not isinstance(coordinates, np.ndarray):
        coordinates = np.array(coordinates)
    
    roi_coords = []
    
    for coord in coordinates:
        # Handle both integer and float coordinates
        y, x = int(round(coord[0])), int(round(coord[1]))
        
        # Check bounds and mask
        if (0 <= y < mask.shape[0] and 
            0 <= x < mask.shape[1] and 
            mask[y, x]):
            roi_coords.append(coord.tolist())  # Convert back to list for consistency
    
    return roi_coords

def calculate_roi_properties(mask, pixel_size_um):
    """Calculate ROI area and volume"""
    area_pixels = np.sum(mask)
    area_um2 = area_pixels * (pixel_size_um ** 2)
    volume_um3 = area_um2 * thickness_um
    return area_pixels, area_um2, volume_um3

def analyze_rois_memory_efficient( masks, coords_spotiflow, pixel_size_um):
    """Process ROIs one at a time to minimize memory usage"""
    num_rois = masks.max()
    results = []
    roi_coords_spotiflow = []
    
    for mask_id in range(1, num_rois + 1):
        single_mask = (masks == mask_id)
        
        roi_spotiflow_coords = assign_spots_to_mask(coords_spotiflow, single_mask)
        spot_count_spotiflow = len(roi_spotiflow_coords)
        
        area_pixels, area_um2, volume_um3 = calculate_roi_properties(single_mask, pixel_size_um)
        
        result = {
            'ROI': mask_id,
            'Spot_Count': spot_count_spotiflow,
            'ROI_Area_pixels': area_pixels,
            'ROI_Area_um2': area_um2,
            'ROI_Volume_um3': volume_um3,
            'Spots_per_Area': spot_count_spotiflow / area_um2 if area_um2 > 0 else 0,
            'Spots_per_Volume': spot_count_spotiflow / volume_um3 if volume_um3 > 0 else 0
        }
        
        results.append(result)
        roi_coords_spotiflow.extend(roi_spotiflow_coords)
        
        del single_mask
    
    return results, roi_coords_spotiflow

In [ ]:
class MicroscopyQC:
    def __init__(self, figsize=(15, 10), dpi=300):
        plt.ioff()
        self.figsize = figsize
        self.dpi = dpi

    def create_qc_figure(self,
                         segmentation_image,
                         spot_std,
                         coordinates,
                         corrected_spots,
                         masks,
                         flow_details,
                         condition,
                         image_num,
                         output_path,
                         pixel_size_um,
                         percentile_range=(1, 99.8),
                         downsample_factor=8,
                         mask_alpha=0.3):
        """
        QC figure using the 'constrained' layout engine for robust, automatic spacing.
        """
        # Create figure using the 'constrained' layout engine
        # This one argument replaces plt.tight_layout() and plt.subplots_adjust()
        fig, axes = plt.subplots(2, 3, figsize=self.figsize,
                                 facecolor='white',
                                 layout='constrained')

        # Calculate intensity limits
        seg_clim = tuple(np.percentile(segmentation_image[::downsample_factor, ::downsample_factor], percentile_range))
        spot_clim = tuple(np.percentile(spot_std[::downsample_factor, ::downsample_factor], percentile_range))
        corrected_clim = tuple(np.percentile(corrected_spots[::downsample_factor, ::downsample_factor], percentile_range))

        axes = axes.flatten()
        for ax in axes:
            ax.axis('off')

        # Panel 1: Segmentation Channel
        axes[0].imshow(segmentation_image, cmap="gray", clim=seg_clim)
        axes[0].set_title('Segmentation Channel', fontsize=12)

        # Panel 2: Spots Channel
        axes[1].imshow(spot_std, cmap="magma", clim=spot_clim)
        axes[1].set_title('Spots Channel', fontsize=12)

        # Panel 3: Spots + Detections
        axes[2].imshow(spot_std, cmap="magma", clim=spot_clim)
        coordinates = np.array(coordinates)
        if len(coordinates) > 0:
            axes[2].scatter(coordinates[:, 1], coordinates[:, 0],
                            facecolors='none', edgecolors='orange', s=10, linewidths=1)
        axes[2].set_title(f'Detections ({len(coordinates)} spots)', fontsize=12)

        # Panel 4: Segmentation + Masks
        axes[3].imshow(segmentation_image, cmap='gray', clim=seg_clim)
        if masks is not None:
            mask_overlay = np.ma.masked_where(masks == 0, masks)
            axes[3].imshow(mask_overlay, alpha=mask_alpha, cmap='tab10', vmin=1)
        axes[3].set_title(f'Masks ({masks.max() if masks is not None else 0} ROIs)', fontsize=12)

        # Panel 5: TopHat Filtered
        axes[4].imshow(corrected_spots, cmap="magma", clim=corrected_clim)
        axes[4].set_title('TopHat Filtered', fontsize=12)

        # Panel 6: Stereographic Flow
        if flow_details is not None:
            try:
                flow_viz = 0.5 * (1 + getattr(flow_details, 'flow', flow_details))
                axes[5].imshow(flow_viz)
            except Exception as e:
                axes[5].text(0.5, 0.5, 'No Flow Data', ha='center', va='center',
                             transform=axes[5].transAxes, fontsize=12, color='gray')
        axes[5].set_title('Stereographic Flow', fontsize=12)

        # Add scale bar
        if pixel_size_um > 0:
            scalebar = ScaleBar(pixel_size_um, units='um', location='lower right',
                                box_alpha=0.8, color='white', box_color='black')
            axes[0].add_artist(scalebar)

        # Set main title
        fig.suptitle(f'{condition} - Image {image_num}', fontsize=16)

        # Save figure
        fig.savefig(output_path, dpi=self.dpi, bbox_inches='tight',
                    facecolor='white', pad_inches=0.1)
        
        plt.close(fig)

In [ ]:
# Find all .czi and .nd2 files 
nd2_files = list(raw_data_folder.glob('*.nd2'))
czi_files = list(raw_data_folder.glob('*.czi'))

# Check what files we have
total_files = len(nd2_files) + len(czi_files)
if total_files == 0:
    raise FileNotFoundError("No .nd2 or .czi files found in raw_data folder")

# Determine file type and print summary
if nd2_files and czi_files:
    raise ValueError("Mixed file types found. Please process .nd2 and .czi files separately.")
elif nd2_files:
    file_type = 'nd2'
    files_to_process = nd2_files
    print(f"Found {len(nd2_files)} .nd2 files to process:")
    for file_path in nd2_files:
        print(f"  - {file_path.name}")
elif czi_files:
    file_type = 'czi'
    files_to_process = czi_files
    print(f"Found {len(czi_files)} .czi files to process:")
    for file_path in czi_files:
        print(f"  - {file_path.name}")

# Group CZI files by condition if needed
if file_type == 'czi':
    condition_groups = {}
    for file_path in files_to_process:
        filename = file_path.stem
        parts = filename.split('_')
        if len(parts) >= 2 and parts[-1].isdigit():
            condition = '_'.join(parts[:-1])
            field_num = int(parts[-1])
        else:
            condition = filename
            field_num = 1
        
        if condition not in condition_groups:
            condition_groups[condition] = []
        condition_groups[condition].append((file_path, field_num))
    
    # Sort files within each condition by field number
    for condition in condition_groups:
        condition_groups[condition].sort(key=lambda x: x[1])

results = []

In [ ]:

if file_type == 'nd2':
    # Process ND2 files
    for file_path in files_to_process:
        condition = file_path.stem
        print(f"\n{'='*50}")
        print(f"Processing {condition}...")
        print(f"{'='*50}")
        
        try:
            array = nd2.imread(file_path, xarray=True, dask=True)
            with nd2.ND2File(file_path) as nd2_file:
                pixel_size_um = nd2_file.voxel_size().x
                num_positions = nd2_file.sizes['P']
                
                for p in tqdm(range(num_positions), desc=f"Processing {condition}"):
                    print(f"\n  Field of view {p+1}/{num_positions}")
                    
                    # Load and process one field of view
                    seg_image, spots_image = process_field_of_view(array, p, channels_params)
                    
                    # Common processing starts here
                    corrected_spots = skimage.morphology.white_tophat(spots_image, footprint=footprint)
                    
                    # Segmentation
                    print("    Running segmentation...")
                    masks = memory_efficient_segmentation(seg_image, segmentation_params)
                    num_rois = masks.max()
                    print(f"    Found {num_rois} gel bags")
                    
                    if num_rois == 0:
                        print("    ⚠️ No ROIs detected in this field of view, skipping analysis and QC.")
                        continue
                    
                    # Count Spots
                    coords_spotiflow, details = detect_spots_spotiflow(corrected_spots)
                    
                    # Analyze ROIs
                    roi_results, coords_spotiflow = analyze_rois_memory_efficient(masks, coords_spotiflow, pixel_size_um)
                    
                    # Add metadata to results
                    for result in roi_results:
                        result.update({
                            'Experiment': experiment_folder.name,
                            'Condition': condition,
                            'Image_Number': p + 1
                        })
                    
                    results.extend(roi_results)
                    
                    # Create QC figure
                    qc_figure_path = figures_folder / f"{condition}_image_{p+1:03d}_QC.png"
                    qc = MicroscopyQC(figsize=(10, 8), dpi=300)
                    qc.create_qc_figure(segmentation_image=seg_image,
                                        spot_std=spots_image,
                                        coordinates=coords_spotiflow,
                                        corrected_spots=corrected_spots,
                                        masks=masks,
                                        flow_details=details.flow,
                                        condition=condition,
                                        image_num=p + 1,
                                        output_path=qc_figure_path,
                                        pixel_size_um=pixel_size_um
                                    )
                    
                    total_spots = len(coords_spotiflow)
                    print(f"    Average spots detected: {total_spots / num_rois}")
                    print(f"    QC figure saved: {qc_figure_path.name}")
                    
                    # Force garbage collection
                    del seg_image, spots_image, corrected_spots, masks, roi_results, coords_spotiflow, details
                    gc.collect()
                    
        except Exception as e:
            print(f"❌ Error processing {condition}: {str(e)}")
            warnings.warn(f"Failed to process {condition}: {str(e)}")

else:  # file_type == 'czi'
    # Process CZI files
    for condition, file_list in condition_groups.items():
        print(f"\n{'='*50}")
        print(f"Processing {condition}...")
        print(f"{'='*50}")
        
        for file_path, field_num in tqdm(file_list, desc=f"Processing {condition}"):
            print(f"\n  Field of view {field_num}")
            
            try:
                # Load CZI file - adjust channel numbers as needed
                czi = CziFile(file_path)
                
                seg_image, seg_info = czi.read_image(C=channels_params['brightfield'])
                spots_image, spots_info = czi.read_image(C=channels_params['spots'])
                
                seg_image = np.squeeze(seg_image)
                spots_image = np.squeeze(spots_image)
                
                seg_image = np.std(seg_image, axis=0)
                del seg_info
    
                spots_image = np.std(spots_image, axis=0)
                del spots_info
                
                img = AICSImage(file_path)
                pixel_size_um = img.physical_pixel_sizes.X
                
                # Common processing starts here (same as ND2)
                corrected_spots = skimage.morphology.white_tophat(spots_image, footprint=footprint)
                
                # Segmentation
                print("    Running segmentation...")
                masks = memory_efficient_segmentation(seg_image, segmentation_params)
                num_rois = masks.max()
                print(f"    Found {num_rois} gel bags")
                
                if num_rois == 0:
                    print("    ⚠️ No ROIs detected in this field of view, skipping analysis and QC.")
                    continue
                
                # Count Spots
                coords_spotiflow, details = detect_spots_spotiflow(corrected_spots)
                
                # Analyze ROIs
                roi_results, coords_spotiflow = analyze_rois_memory_efficient(masks, coords_spotiflow, pixel_size_um)
                
                # Add metadata to results
                for result in roi_results:
                    result.update({
                        'Experiment': experiment_folder.name,
                        'Condition': condition,
                        'Image_Number': field_num
                    })
                
                results.extend(roi_results)
                
                # Create QC figure
                qc_figure_path = figures_folder / f"{condition}_image_{field_num:03d}_QC.png"
                qc = MicroscopyQC(figsize=(10, 8), dpi=300)
                qc.create_qc_figure(segmentation_image=seg_image,
                                    spot_std=spots_image,
                                    coordinates=coords_spotiflow,
                                    corrected_spots=corrected_spots,
                                    masks=masks,
                                    flow_details=details.flow,
                                    condition=condition,
                                    image_num=field_num,
                                    output_path=qc_figure_path,
                                    pixel_size_um=pixel_size_um
                                )
                
                total_spots = len(coords_spotiflow)
                print(f"    Average spots detected: {total_spots / num_rois}")
                print(f"    QC figure saved: {qc_figure_path.name}")
                
                # Force garbage collection
                del seg_image, spots_image, corrected_spots, masks, roi_results, coords_spotiflow, details
                gc.collect()
                
            except Exception as e:
                print(f"❌ Error processing {file_path.name}: {str(e)}")
                warnings.warn(f"Failed to process {file_path.name}: {str(e)}")

print(f"\n🎉 Processing complete! Analyzed {len(results)} ROIs total.")

In [ ]:
# Cell 7: Save results to files
if not results:
    print("❌ No results to save")
else:
    df = pd.DataFrame(results )
    
    # Filter out ROIs with area < 3000 µm²
    df = df[df['ROI_Area_um2'] >= 3000]
    
    # Save as CSV
    csv_path = processed_data_folder / f"{experiment_folder.name}_results.csv"
    df.to_csv(csv_path, index=False)
    
    # Save as Excel with multiple sheets
    excel_path = processed_data_folder / f"{experiment_folder.name}_results.xlsx"
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='All_Data', index=False)
        
        # Summary by condition
        summary = df.groupby('Condition').agg({
            'Spot_Count': ['count', 'mean', 'std', 'sum'],
            'ROI_Area_um2': ['mean', 'std'],
            'Spots_per_Area': ['mean', 'std'],
            'Spots_per_Volume': ['mean', 'std']
        }).round(3)
        
        summary.columns = ['_'.join(col).strip() for col in summary.columns]
        summary.reset_index().to_excel(writer, sheet_name='Summary_by_Condition', index=False)
    
    print(f"✅ Results saved to:")
    print(f"   📄 CSV: {csv_path}")
    print(f"   📊 Excel: {excel_path}")
    
    # Display basic statistics
    print(f"\n📈 Quick Statistics:")
    print(f"   Total ROIs analyzed: {len(df)}")
    print(f"   Conditions: {df['Condition'].nunique()}")
    print(f"   Total spots detected: {df['Spot_Count'].sum()}")
    print(f"   Average spots per ROI: {df['Spot_Count'].mean():.1f} ± {df['Spot_Count'].std():.1f}")

In [ ]:
# Cell 8: Preview results
# Display first few rows and basic info
if results:
    df = pd.DataFrame(results)
    # Filter out ROIs with area < 3000 µm²
    df = df[df['ROI_Area_um2'] >= 3000]
    print("📋 Results Preview:")
    print(df.head(5))
    
    print(f"\n📊 Summary by Condition:")
    condition_summary = df.groupby('Condition').agg({
        'ROI': 'count',
        'Spot_Count': ['mean', 'std'],
        'ROI_Area_um2': ['mean', 'std'],
        'Spots_per_Area': ['mean', 'std']
    }).round(3)
    condition_summary.columns = ['_'.join(col).strip() for col in condition_summary.columns]
    print(condition_summary)

In [ ]:
# Cell 9: Create summary analysis figures
if results:
    df = pd.DataFrame(results)
    # Filter out ROIs with area < 3000 µm²
    df = df[df['ROI_Area_um2'] >= 3000]
    
    # Calculate sample sizes for each condition
    sample_sizes = df.groupby('Condition').size()
    
    # Set up the plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Figure 1: Spot counts by condition
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Function to add sample size annotations
    def add_n_annotations(ax, data_col, group_col='Condition'):
        """Add sample size annotations to boxplot"""
        # Get unique conditions and their positions
        conditions = df[group_col].unique()
        
        for i, condition in enumerate(conditions):
            n = len(df[df[group_col] == condition])
            # Add annotation to the right of each boxplot
            ax.text(ax.get_xlim()[1] * 0.98, i, f'n={n}', 
                   verticalalignment='center', 
                   horizontalalignment='right',
                   fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    # Spot count distribution
    sns.boxplot(data=df, y='Condition', x='Spot_Count', ax=axes[0, 0])
    axes[0, 0].set_title('Spot Count Distribution by Condition')
    axes[0, 0].set_ylabel('')
    add_n_annotations(axes[0, 0], 'Spot_Count')
    
    # ROI area distribution
    sns.boxplot(data=df, y='Condition', x='ROI_Area_um2', ax=axes[0, 1])
    axes[0, 1].set_title('ROI Area Distribution by Condition')
    axes[0, 1].set_xlabel('ROI Area (μm²)')
    axes[0, 1].set_ylabel('')
    add_n_annotations(axes[0, 1], 'ROI_Area_um2')
    
    # Spots per area
    sns.boxplot(data=df, y='Condition', x='Spots_per_Area', ax=axes[1, 0])
    axes[1, 0].set_title('Spots per Area by Condition')
    axes[1, 0].set_xlabel('Spots per μm²')
    axes[1, 0].set_ylabel('')
    add_n_annotations(axes[1, 0], 'Spots_per_Area')
    
    # Spots per volume
    sns.boxplot(data=df, y='Condition', x='Spots_per_Volume', ax=axes[1, 1])
    axes[1, 1].set_title('Spots per Volume by Condition')
    axes[1, 1].set_xlabel('Spots per μm³')
    axes[1, 1].set_ylabel('')
    add_n_annotations(axes[1, 1], 'Spots_per_Volume')
    
    plt.tight_layout()
    summary_path = figures_folder / 'Summary_Analysis.png'
    plt.savefig(summary_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Summary figure saved: {summary_path}")

In [ ]:
# Cell 10: Create correlation analysis figures
if results:
    df = pd.DataFrame(results)
    # Filter out ROIs with area < 3000 µm²
    df = df[df['ROI_Area_um2'] >= 3000]
    
    # Figure 2: Correlation plot
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    
    # Spot count vs ROI area
    sns.scatterplot(data=df, x='ROI_Area_um2', y='Spot_Count', hue='Condition', ax=ax)
    ax.set_title('Spot Count vs ROI Area')
    ax.set_xlabel('ROI Area (μm²)')
    ax.set_ylabel('Spot Count')
    
    plt.tight_layout()
    correlation_path = figures_folder / 'Correlation_Analysis.png'
    plt.savefig(correlation_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Correlation figure saved: {correlation_path}")